# 🐼 Pandas 完整学习手册（带实战代码）

本 Notebook 系统梳理了 pandas 的核心知识点，每部分包含：
- 概念说明
- 实战代码（带详细注释）
- 可直接运行验证

建议逐节运行并修改参数加深理解。

In [ ]:
# 导入必要库
import pandas as pd
import numpy as np

# 设置显示选项（可选）
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. 基础入门：DataFrame 与 Series

In [ ]:
# 创建 Series
s = pd.Series([1, 3, 5, np.nan, 6])
print("Series 示例:")
print(s)

# 创建 DataFrame（从字典）
df = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'city': ['NY', 'LA', 'Chicago']
})
print("\nDataFrame 示例:")
print(df)

# 查看基本信息
print("\n--- 基本信息 ---")
print("形状 (行, 列):", df.shape)
print("\n前两行:")
print(df.head(2))
print("\n数据类型:")
print(df.dtypes)
print("\n统计摘要:")
print(df.describe(include='all'))

## 2. 数据读取与写入

In [ ]:
# 创建示例数据并保存为 CSV
sample_data = pd.DataFrame({
    'id': [1, 2, 3],
    'value': [10.5, 20.1, 30.0],
    'date': ['2023-01-01', '2023-01-02', '2023-01-03']
})

# 写入 CSV（不保存索引）
sample_data.to_csv('sample.csv', index=False)

# 读取 CSV（自动解析日期）
df_read = pd.read_csv('sample.csv', parse_dates=['date'])
print("从 CSV 读取的数据:")
print(df_read)
print("\n日期列类型:", df_read['date'].dtype)

## 3. 索引与选择数据

In [ ]:
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': ['x', 'y', 'z', 'w'],
    'C': [10, 20, 30, 40]
}, index=['row1', 'row2', 'row3', 'row4'])

print("原始 DataFrame:")
print(df)

# 列选择
print("\n选择列 A:", df['A'].tolist())
print("选择多列:\n", df[['A', 'C']])

# 行选择：iloc（位置） vs loc（标签）
print("\n.iloc[0:2]（前两行）:\n", df.iloc[0:2])
print("\n.loc['row1':'row3']（标签切片）:\n", df.loc['row1':'row3'])

# 布尔索引
print("\nA > 2 的行:\n", df[df['A'] > 2])

# query 方法（更简洁）
print("\nquery('C > 25'):\n", df.query('C > 25'))

## 4. 数据清洗

In [ ]:
df_dirty = pd.DataFrame({
    'name': ['Alice', 'Bob', None, 'David'],
    'age': [25, np.nan, 30, 35],
    'score': [85, 90, 90, None]
})

print("原始脏数据:\n", df_dirty)

# 检查缺失值
print("\n缺失值统计:\n", df_dirty.isna().sum())

# 填充缺失值（数值列用均值，字符列用众数）
df_clean = df_dirty.copy()
df_clean['age'].fillna(df_clean['age'].mean(), inplace=True)
df_clean['name'].fillna(df_clean['name'].mode()[0], inplace=True)  # mode() 返回 Series
df_clean['score'].fillna(-1, inplace=True)  # 用 -1 标记缺失

print("\n清洗后:\n", df_clean)

# 去重
df_dup = pd.DataFrame({'A': [1, 1, 2], 'B': ['x', 'x', 'y']})
print("\n去重前:\n", df_dup)
print("去重后:\n", df_dup.drop_duplicates())

## 5. 数据变换与新增列

In [ ]:
df = pd.DataFrame({
    'product': ['A', 'B', 'C'],
    'price': [100, 150, 200],
    'quantity': [10, 5, 8]
})

# 方法1：直接赋值
df['total'] = df['price'] * df['quantity']

# 方法2：使用 assign（链式操作友好）
df = df.assign(discount=lambda x: x['total'] * 0.1)

# 方法3：apply 自定义函数
df['category'] = df['price'].apply(lambda p: 'High' if p > 150 else 'Low')

# 条件赋值：np.where
df['flag'] = np.where(df['quantity'] > 7, 'Popular', 'Normal')

print("变换后 DataFrame:\n", df)

## 6. 分组与聚合（GroupBy）

In [ ]:
sales = pd.DataFrame({
    'region': ['North', 'South', 'North', 'South', 'East'],
    'product': ['A', 'A', 'B', 'B', 'A'],
    'amount': [100, 150, 200, 120, 90]
})

print("销售数据:\n", sales)

# 基础分组聚合
print("\n按 region 求和:\n", sales.groupby('region')['amount'].sum())

# 多列聚合
print("\n多指标聚合:\n", sales.groupby('region').agg({
    'amount': ['sum', 'mean'],
    'product': 'count'
}))

# transform：广播回原 DataFrame
sales['region_avg'] = sales.groupby('region')['amount'].transform('mean')
print("\n添加区域平均值后:\n", sales)

## 7. 合并与连接

In [ ]:
df1 = pd.DataFrame({'id': [1, 2, 3], 'name': ['Alice', 'Bob', 'Charlie']})
df2 = pd.DataFrame({'id': [1, 2, 4], 'salary': [5000, 6000, 7000]})

print("df1:\n", df1)
print("\ndf2:\n", df2)

# 内连接（交集）
inner = pd.merge(df1, df2, on='id', how='inner')
print("\n内连接:\n", inner)

# 左连接（保留 df1 所有行）
left = pd.merge(df1, df2, on='id', how='left')
print("\n左连接:\n", left)

# 纵向拼接（concat）
df3 = pd.DataFrame({'id': [4, 5], 'name': ['David', 'Eve']})
concatenated = pd.concat([df1, df3], ignore_index=True)
print("\n纵向拼接:\n", concatenated)

## 8. 重塑与透视表

In [ ]:
# 宽表转长表（melt）
wide = pd.DataFrame({
    'product': ['A', 'B'],
    'Q1': [100, 150],
    'Q2': [120, 160]
})
long = wide.melt(id_vars='product', var_name='quarter', value_name='sales')
print("宽表:\n", wide)
print("\n长表（melt）:\n", long)

# 透视表（长表转宽表）
pivot = long.pivot(index='product', columns='quarter', values='sales')
print("\n透视表:\n", pivot)

# pivot_table 支持聚合（处理重复）
long_dup = pd.concat([long, long])  # 制造重复
pivot_agg = pd.pivot_table(long_dup, values='sales', index='product', columns='quarter', aggfunc='mean')
print("\n带聚合的透视表:\n", pivot_agg)

## 9. 时间序列处理

In [ ]:
# 创建时间序列数据
dates = pd.date_range('2023-01-01', periods=10, freq='D')
ts = pd.DataFrame({
    'date': dates,
    'value': np.random.randn(10).cumsum()
})
ts.set_index('date', inplace=True)

print("时间序列数据:\n", ts.head())

# 重采样（日 → 周）
weekly = ts.resample('W').mean()
print("\n周均值:\n", weekly)

# 移动窗口（7天滚动均值）
ts['rolling_mean'] = ts['value'].rolling(window=3).mean()
print("\n带滚动均值:\n", ts.head())

## 10. 链式操作与最佳实践

In [ ]:
# 示例：链式数据处理
result = (pd.DataFrame({
            'category': ['A', 'B', 'A', 'B', 'A'],
            'value': [10, 20, 30, 40, 50]
        })
        .query('value > 15')
        .assign(ratio=lambda x: x['value'] / x['value'].sum())
        .groupby('category')
        .agg(total=('value', 'sum'), count=('value', 'size'))
        .reset_index()
       )

print("链式操作结果:\n", result)

## 🎉 完成！

你现在拥有一份完整的 pandas 学习手册！

**建议：**
- 修改代码中的数据，观察输出变化
- 尝试用自己的数据集实践
- 将此 Notebook 作为日后参考